In [ ]:
# pip install Scrapy
# 특정 시나리오를 작성해서 데이터를 추출
# 페이지 파싱, 저장 등의 각 기능을 분리해서 작성한다.
# 같은 코드를 똑같이 작성하는 낭비를 막고 싶을 때 Scrapy를 사용한다.
# 어떤 데이터를 추출하고, 어디에 저장할 것인지만 작성

# 1. scrapy 명령어를 사용해 프로젝트를 만든다.
# 2. Spider 클래스를 작성해서 크롤링과 데이터 추출 코드 작성
# 3. 명령줄에서 scary명령어를 실행

# 1. scrapy 프로젝트 생성(프로젝트를 만들 폴더로 이동한다.)
# c:\> scrapy startproject wiki

# 2. spider 클래스 만들기
# Scrapy 프레임워크에서는 Spider클래스를 상속해서 서브클래스를 만들며, 서브클래스에서 다양한 처리를 하게 된다.
# book1이라는 이름의 서브클래스르 만든다. 
# wiki
#   |-- spiders
#     |--book1.py
import scrapy

class Book1Spider(scrapy.Spider):
  name = 'book'
  start_urls = [
    'https://wikibook.co.kr/list/' # 추출 대상인 URL지정 
  ]

  def parse(self, response):
    # 도서 목록 추출하기 --- ( ※ 1)
    title = response.css('title') # css()메서드를 이용하여 DOM을 추출
    print(title.extract()) #   DOM에 있는 데이터를 추출한다.


# 3. 명령 줄에서 실행해서 결과 추출
# c:\>Scrapy crawl book --nolog    
# ['<title>도서 목록 | 위키북스</title>']  

#4. 데이터 추출해서 저장 : 위 코드를 수정
# wiki
#   |-- spiders
#     |--book2.py
import scrapy

class Book2Spider(scrapy.Spider):
  name = 'book2'
  start_urls = [
    'https://wikibook.co.kr/list/'
  ]

  def parse(self, response):
    # 도서 목록 추출하기 --- ( ※ 1)
    li_list = response.css('.book-url')
    for a in li_list:
      # href 속성과 텍스트 추출하기 --- ( ※ 2)
      href = a.css('::attr(href)').extract_first() # 결과에 포함된 첫번째 요소를 리턴 한다.
      text = a.css('::text').extract_first()
      # 절대 경로로 변환하기 --- ( ※ 3)
      href2 = response.urljoin(href) # https://wikibook.co.kr/refactoring-databases : href에 있는 값을 붙여서 절대경로로.
      # 결과 내기 --- ( ※ 4)
      yield {   # {'text': '실천가를 위한 실용주의 프로젝트 관리', 'url': 'https://wikibook.co.kr/behind-closed-doors'} 리턴
        'text': text,
        'url': href2
      }
# c:\>Scrapy crawl book2 -o list.json
# 실행하면 list.json파일이 생성


In [47]:
import json, pprint
f = open("wiki\wiki\spiders\list.json")
data = json.load(f)
pprint.pprint(data)
###########################################################
# c:\>crapy shell
# fetch("https://wikibook.co.kr/list/")
# response.css("title::text").ectract_first()
# response.css("h4::text").extract_first()


[{'text': '만들면서 배우는 클린 아키텍처',
  'url': 'https://wikibook.co.kr/clean-architecture'},
 {'text': '머신러닝 시스템 디자인 패턴', 'url': 'https://wikibook.co.kr/mlsdp'},
 {'text': '모든 기획자와 디자이너가 알아야 할 사람에 대한 100가지 사실(개정판)',
  'url': 'https://wikibook.co.kr/100things2ed'},
 {'text': '파이썬 데이터 클리닝 쿡북', 'url': 'https://wikibook.co.kr/data-cleansing'},
 {'text': '따라하며 배우는 도커와 CI 환경', 'url': 'https://wikibook.co.kr/docker-ci'},
 {'text': '기획자의 글쓰기', 'url': 'https://wikibook.co.kr/on-planners-writing'},
 {'text': '세상 모든 디자인, 미리캔버스 하나로!', 'url': 'https://wikibook.co.kr/miricanvas'},
 {'text': '메타버스로 가는 티켓, 게더타운의 모든 것',
  'url': 'https://wikibook.co.kr/gathertown'},
 {'text': '나의 첫 블렌더', 'url': 'https://wikibook.co.kr/blender'},
 {'text': '수학으로 풀어보는 강화학습 원리와 알고리즘 (개정판)',
  'url': 'https://wikibook.co.kr/mathrlrev'},
 {'text': '비트코인·블록체인 바이블', 'url': 'https://wikibook.co.kr/bcbc'},
 {'text': 'Real MySQL 8.0 [2권]', 'url': 'https://wikibook.co.kr/realmysql802'},
 {'text': 'Real MySQL 8.0 [1권]', 'url': 'https://wi

In [ ]:
# c:/>scrapy genspider book3 https://wikibook.co.kr 명령어를 실행시키면 book3.py파일이 생성된다. 
# wiki
#   |-- spiders
#     |--book3.py
class Book3Spider(scrapy.Spider):
    name = 'book3'
    allowed_domains = ['https://wikibook.co.kr']
    start_urls = ['http://https://wikibook.co.kr/']  # 크롤링을 시작할 url지정
    
    # 도서 목로에서 개별 도서목록의 링크를 추출, 개별도서페이지크롤링을 지시하는 함수
    def parse(self, response):
        li_list = response.css('.book-url')
        for a in li_list[:5]:
          href = a.css('::attr(href)').extract_first()
          print(href)
          # 개별 도서 페이지 크롤링 요청하기 --- (※3)
          yield response.follow(
            response.urljoin(href), self.parse_book
          )
    # 개별 도서 페이지를 스크레이핑 하는 함수
    def parse_book(self, response):
    # 제목과 링크 추출하기 --- (※5) 도서 페이지에서 이름과 이미지 링크
    title = response.css('.main-title::text').extract_first()
    img_url = response.css('.book-image-2d::attr(src)').extract_first()
    # 결과 출력하기 --- (※6)
    yield {                                                       #  yield : [jiːld]
      'title': title,
      'img_url': response.urljoin(img_url)
    }

    # c:\>scrapy crawl book3 -o list3.json

In [1]:
import json, pprint
f = open("wiki\wiki\spiders\list3.json")
data = json.load(f)
pprint.pprint(data)

[{'img_url': 'https://wikibook.co.kr/images/cover/s/9791158392758.jpg',
  'title': '만들면서 배우는 클린 아키텍처'},
 {'img_url': 'https://wikibook.co.kr/images/cover/s/9791158392765.jpg',
  'title': '모든 기획자와 디자이너가 알아야 할 사람에 대한 100가지 사실(개정판)'},
 {'img_url': 'https://wikibook.co.kr/images/cover/s/9791158392789.jpg',
  'title': '파이썬 데이터 클리닝 쿡북'},
 {'img_url': 'https://wikibook.co.kr/images/cover/s/9791158392888.jpg',
  'title': '머신러닝 시스템 디자인 패턴'},
 {'img_url': 'https://wikibook.co.kr/images/cover/s/9791158392864.jpg',
  'title': '따라하며 배우는 도커와 CI 환경'}]


In [ ]:
# c:/>scrapy genspider book4 https://wikibook.co.kr
import scrapy

class Book4Spider(scrapy.Spider):
  name = 'book4'
  allowed_domains = ['wikibook.co.kr']
  start_urls = [
    'https://wikibook.co.kr/list'
  ]

  def parse(self, response):
    li_list = response.css('.book-url')
    # 위키북스 사이트에 부담을 주지 않게 5개만 다운로드 합니다.
    for a in li_list[:5]:
      href = a.css('::attr(href)').extract_first()
      yield response.follow(
        response.urljoin(href), self.parse_book
      )

    def parse_book(self, response):
        title = response.url.split("/")[-2]
        img_url = response.css('.book-image-2d::attr(src)').extract_first()
        # 다운로드를 지시합니다. --- (※1)  
        # scrapy.Request() 사용, 첫번째 매개변수는 URL, 두번째 매개변수는 크롤링완료했을때 처리할 내용
        req = scrapy.Request(
          response.urljoin(img_url),
          callback=self.parse_img 
        )
        # 함수끼리 데이터를 전송하는 방법입니다. --- (※2) 
        
        req.meta["title"] = title
        yield req

  # 이미지 다운로드 --- (※3) 이미지 크롤링이 완료되면 콜백에 지정한 메서드가 실행 parse_img()메서드
  def parse_img(self, response):
    # 전달된 데이터를 받습니다. --- (※4)
    title = response.meta["title"]
    title = title.replace(r'[\/:*?"<>|.]+', '_').strip()
    fname = title + '.jpg'
    # 파일을 저장합니다. --- (※5)
    with open(fname, 'wb') as f:
      f.write(response.body)

In [ ]:
# c:\>scrapy startprojct coupang
# scrapy genspider coupang1 coupang.com
# coupang\coupang폴더에 selenium_middleware.py파일 생성


# selenium_middleware.py
from scrapy.http import HtmlResponse
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Chrome 초기화하기 --- ( ※ 1)
driver = webdriver.Chrome(ChromeDriverManager().install())

# Chrome로 URL 열기 --- ( ※ 2) 
def selenium_get(url): # 
    driver.get(url)

# CSS 쿼리를 지정해서 읽어들일 때까지 대기하기 --- ( ※ 3) 
def get_dom(query):
    dom = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
          (By.CSS_SELECTOR, query)))
  return dom

# Firefox 종료하기 --- ( ※ 4)
def selenium_close():
  driver.close()

# 미들웨어 --- ( ※ 5)
class SeleniumMiddleware(object):
  # 요청을 Selenium으로 처리하기 --- ( ※ 6)
  def process_request(self, request, spider):
    driver.get(request.url)
    return HtmlResponse(
      driver.current_url,
      body = driver.page_source,
      encoding = 'utf-8',
      request = request)

In [ ]:
# coupang1.py파일 편집

import scrapy
# Selenium 미들웨어 읽어 들이기
from ..selenium_middleware import *

# 쿠팡 이메일과 비밀번호 지정하기 --- (※1)
USER = "쿠팡 이메일을 입력해주세요"
PASSWORD = "쿠팡 비밀번호를 입력해주세요"

class CoupangSpider(scrapy.Spider):
  name = 'coupang1'
  # 미들웨어 등록하기 --- (※2)
  custom_settings = {
    "DOWNLOADER_MIDDLEWARES": {
      "coupang.selenium_middleware.SeleniumMiddleware": 0
    }
  }

  # 요청 전에 로그인하기 --- (※3)
  def start_requests(self):
    # 로그인 페이지로 이동 후 로그인
    selenium_get("https://login.coupang.com/login/login.pang")
    email = get_dom('._loginForm [name=email]')
    email.send_keys(USER)
    password = get_dom('._loginForm [name=password]')
    password.send_keys(PASSWORD)
    button = get_dom("._loginForm button[type=submit]")
    button.click()

    # 마이 페이지로 이동
    a = get_dom('#myCoupang > a')
    mypage = a.get_attribute('href')
    yield scrapy.Request(mypage, self.parse)
  
  def parse(self, response):
    # 원하는 정보 추출하기 --- (※4)
    #items = response.css('.my-order-unit__item-info')
    items = response.css('.sc-9cwg9-6')
    for item in items:
      #title = item.css(".my-order-unit__info-name strong:last-child::text").extract_first().strip()
      title = item.css(".sc-755zt3-1::text").extract_first().strip()
      #info = item.css(".my-order-unit__info-ea::text").extract_first().split("/")[0].strip()
      info = item.css(".sc-755zt3-0::text").extract_first().split("/")[0].strip()
      yield {
        "title": title,
        "info": info
      }

In [ ]:
# 3 장

In [3]:
from bs4 import BeautifulSoup 
import urllib.request as req
import os.path
url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
savename = "forecast.xml"   #--- (※1) xml 파일을 내려 받기
if not os.path.exists(savename):
    req.urlretrieve(url, savename)
# BeautifulSoup로 분석하기 --- (※2) 
xml = open(savename, "r", encoding="utf-8").read()
soup = BeautifulSoup(xml, 'html.parser') # BeautifulSoup으로 xml파일 분석하기
print(soup) # html.parser는 html을 분석하기 위해서 만들어 짐
# 각 지역 확인하기 --- (※3)
info = {}
for location in soup.find_all("location"):  # 모두
    name = location.find('city').string    # 해당 하는 값 한개
    weather = location.find('wf').string 
    if not (weather in info):
        info[weather] = []
    info[weather].append(name)
# 각 지역의 날씨를 구분해서 출력하기
for weather in info.keys():
    print("+", weather)
    for name in info[weather]:
        print("| - ", name)

<?xml version="1.0" encoding="utf-8" ?>
<rss version="2.0">
<channel>
<title>기상청 육상 중기예보</title>
<link/>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp
<description>기상청 날씨 웹서비스</description>
<language>ko</language>
<generator>기상청</generator>
<pubdate>2021년 11월 10일 (수)요일 06:00</pubdate>
<item>
<author>기상청</author>
<category>육상중기예보</category>
<title>전국 육상 중기예보 - 2021년 11월 10일 (수)요일 06:00 발표</title>
<link/>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp
<guid>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp</guid>
<description>
<header>
<title>전국 육상중기예보</title>
<tm>202111100600</tm>
<wf><![CDATA[○ (강수) 14일(일) 오후에는 수도권과 강원영서에 비가 오겠고, 강원영서는 15일(월)까지 이어지겠습니다.   <br />○ (기온) 이번 예보기간 아침 기온은 -2~11도로 어제(9일, 아침 기온 3~7도)와 비슷하거나 조금 낮겠고, 낮 기온은 10~18도로 어제(낮 기온 7~14도)보다 조금 높겠습니다. <br />○ (주말전망) 13일(토)~14일(일) 오전까지 전국이 가끔 구름많다가 14일(일) 오후에는 수도권과 강원영서에 비가 오겠습니다. 아침 기온은 -2~10도, 낮 기온은 13~18도가 되겠습니다.]]></wf>
</header>
<body>
<location wl_ver="3">
<province>서울ㆍ인천ㆍ경기도</province>
<city>서울</city

In [4]:
import urllib.request as req
import os.path, random
import json
# JSON 데이터 내려받기 --- (※1)
url = "https://api.github.com/repositories"
savename = "repo.json"
if not os.path.exists(savename):
    req.urlretrieve(url, savename)
# JSON 파일 분석하기 --- (※2)
# items = open(savename, "r", encoding="utf-8")
# print(items.read())
items = json.load(open(savename, "r", encoding="utf-8"))
print(items)
# 또는
# s = open(savename, "r", encoding="utf-8").read()
# items = json.loads(s)
# 출력하기 --- (※3)
for item in items:
    print(item["name"] + " - " + item["owner"]["login"])

[{'id': 1, 'node_id': 'MDEwOlJlcG9zaXRvcnkx', 'name': 'grit', 'full_name': 'mojombo/grit', 'private': False, 'owner': {'login': 'mojombo', 'id': 1, 'node_id': 'MDQ6VXNlcjE=', 'avatar_url': 'https://avatars.githubusercontent.com/u/1?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/mojombo', 'html_url': 'https://github.com/mojombo', 'followers_url': 'https://api.github.com/users/mojombo/followers', 'following_url': 'https://api.github.com/users/mojombo/following{/other_user}', 'gists_url': 'https://api.github.com/users/mojombo/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/mojombo/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/mojombo/subscriptions', 'organizations_url': 'https://api.github.com/users/mojombo/orgs', 'repos_url': 'https://api.github.com/users/mojombo/repos', 'events_url': 'https://api.github.com/users/mojombo/events{/privacy}', 'received_events_url': 'https://api.github.com/users/mojombo/received_events', 'type': 'U

In [5]:
import json
price = {"date":"2020-10-10",
	"price":{"Apple":80, 
			"Orange":55,
			"Banana":40
			}}
print(price) # 딕셔너리로 출력
s = json.dumps(price)  # json으로 변환
print(s)
# json과 딕셔너리가 같다.

{'date': '2020-10-10', 'price': {'Apple': 80, 'Orange': 55, 'Banana': 40}}
{"date": "2020-10-10", "price": {"Apple": 80, "Orange": 55, "Banana": 40}}


In [17]:
import yaml 
# YAML 정의하기 ---- (※1) 들여쓰기를 사용해 계층 구조를 표현
# XML보다 간단하고 json과 거의 비슷하다. YAML과 JSON은 대 용량으로 많이 사용한다.

# yaml으로 문자열 정의
yaml_str = """   
Date: 2017-03-10
PriceList:
    -
        item_id: 1000
        name: Banana
        color: yellow
        price: 800
    -
        item_id: 1001
        name: Orange
        color: orange
        price: 1400
    -
        item_id: 1002
        name: Apple
        color: red
        price: 2400
"""
# YAML 분석하기 --- (※2) YAML문자열 데이터를 분석하고 파이썬의 데이터로 변환한다.
data = yaml.load(yaml_str,Loader=yaml.FullLoader) # JSON형식으로 저장한다. 리스트에 딕셔러니
# 이름과 가격 출력하기 --- (※3)
print(data['PriceList'])
for item in data['PriceList']:
    print(item["name"], item["price"])

[{'item_id': 1000, 'name': 'Banana', 'color': 'yellow', 'price': 800}, {'item_id': 1001, 'name': 'Orange', 'color': 'orange', 'price': 1400}, {'item_id': 1002, 'name': 'Apple', 'color': 'red', 'price': 2400}]
Banana 800
Orange 1400
Apple 2400


In [23]:
import yaml
# 파이썬 데이터를 YAML 데이터로 출력하기
customer = [
    { "name": "InSeong", "age": "24", "gender": "man" },
    { "name": "Akatsuki", "age": "22", "gender": "woman" },
    { "name": "Harin", "age": "23", "gender": "man" },
    { "name": "Yuu", "age": "31", "gender": "woman" }
]
# 파이썬 데이터를 YAML 데이터로 변환하기
yaml_str = yaml.dump(customer)
print(yaml_str)
print("--- --- ---")
# YAML 데이터를 파이썬 데이터로 변환하기
# data = yaml.safe_load(yaml_str)
# data = yaml.load(yaml_str, Loader=yaml.CLoader)
data = yaml.load(yaml_str,Loader=yaml.FullLoader)
# 이름 출력하기
for p in data:
    print(p["name"])

- age: '24'
  gender: man
  name: InSeong
- age: '22'
  gender: woman
  name: Akatsuki
- age: '23'
  gender: man
  name: Harin
- age: '31'
  gender: woman
  name: Yuu

--- --- ---
InSeong
Akatsuki
Harin
Yuu


In [24]:
import yaml
# 문자열로 YAML을 정의합니다.
yaml_str = """
# 정의
color_def:
  - &color1 "#FF0000"
  - &color2 "#00FF00"
  - &color3 "#0000FF"
# 별칭 테스트
color:
  title: *color1
  body: *color2
  link: *color3
"""
# YAML 데이터 분석하기
data = yaml.load(yaml_str,Loader=yaml.FullLoader)
# 별칭이 전개됐는지 테스트하기
print("title=", data["color"]["title"])
print("body=", data["color"]["body"])
print("link=", data["color"]["link"])

title= #FF0000
body= #00FF00
link= #0000FF


In [3]:
import csv, codecs
# CSV 파일 열기
filename = "list-euckr.csv"
fp = codecs.open(filename, "r", "utf-8") # ansi인 경우 euc_kr로 지정
# 한 줄씩 읽어 들이기
reader = csv.reader(fp, delimiter=",", quotechar='"')
for cells in reader:
    print(cells[1], cells[2])

이름 가격
비누 300
장갑 150
마스크 230


In [4]:
import csv, codecs
with codecs.open("test.csv", "w", "euc_kr ") as fp:
    writer = csv.writer(fp, delimiter=",", quotechar='"')
    writer.writerow(["ID", "이름", "가격"])
    writer.writerow(["1000", "SD 카드 ", 30000])
    writer.writerow(["1001", "키보드", 21000])
    writer.writerow(["1002", "마우스", 15000])

In [38]:
import xlrd
#import openpyxl 
# 엑셀 파일 열기く --- (※1)
filename = "stat_104102.xls"
#filename = "stat_104102.xlsx"
#book = openpyxl.load_workbook(filename)
book = xlrd.open_workbook(filename)
# 맨 앞의 시트 추출하기 --- (※2)
#sheet = book.worksheets[0]
sheet = book.sheet_by_index(0)

# 시트의 각 행을 순서대로 추출하기 --- (※3)
data = []
for row in range(sheet.nrows):
    data.append([
        sheet.row_values(row)[0],
        sheet.row_values(row)[9]
    ])
#     data.append([
#         row[0].value,   #--- 첫 번째 컬럼
#         row[9].value   # --- 10번째 컬럼
#     ])
# 데이터를 인구 순서로 정렬합니다.
# # 필요없는 줄(헤더, 연도, 계) 제거하기
# del data[0]     #---첫번째 행 삭제
# del data[1]     #---두번째 행 삭제
# del data[2]     #---세번째 행 삭제

data = sorted(data, key=lambda x:x[1])
print(data)
# 필요없는 줄(헤더, 연도, 계) 제거하기
del data[0]     #---첫번째 행 삭제
del data[0]     #---두번째 행 삭제
del data[0]     #---세번째 행 삭제
del data[0]     #---네번째 행 삭제
# 하위 5위를 출력합니다.

for i, a in enumerate(data):
    if (i >= 5): break
    print(i+1, a[0], a[1])

[['통계표명:', ''], ['단위:', ''], ['출처:', ''], ['주석:', ''], ['울산', '1,148'], ['광주', '1,456'], ['대전', '1,475'], ['강원', '1,542'], ['충북', '1,600'], ['전북', '1,819'], ['전남', '1,869'], ['경기', '13,240'], ['충남', '2,124'], ['대구', '2,438'], ['경북', '2,666'], ['인천', '2,957'], ['', '2019'], ['경남', '3,363'], ['부산', '3,414'], ['세종', '341'], ['계', '51,850'], ['제주', '671'], ['서울', '9,729']]
1 울산 1,148
2 광주 1,456
3 대전 1,475
4 강원 1,542
5 충북 1,600


In [2]:
import openpyxl 
# 엑셀 파일 열기 --- (※1)
filename = "stat_104102.xlsx"
book = openpyxl.load_workbook(filename)
# 활성화된 시트 추출하기 --- (※2)
sheet = book.active
# 서울을 제외한 인구를 구해서 쓰기 --- (※3)
for i in range(0, 9):
    total = int(sheet[str(chr(i + 66)) + "3"].value.replace(",",""))
    seoul = int(sheet[str(chr(i + 66)) + "4"].value.replace(",",""))
    output = total - seoul
    print("서울 제외 인구 =", output)
    # 쓰기 --- (※4)
    sheet[str(chr(i + 66)) + "21"] = output
    cell = sheet[str(chr(i + 66)) + "21"]
    # 폰트와 색상 변경해보기 --- (※5)
    cell.font = openpyxl.styles.Font(size=14,color="FF0000")
    cell.number_format = cell.number_format
# 엑셀 파일 저장하기 --- (※6)
filename = "population.xlsx"
book.save(filename)
print("ok")

서울 제외 인구 = -48723
서울 제외 인구 = -48936
서울 제외 인구 = -49128
서울 제외 인구 = -49314
서울 제외 인구 = -49514
서울 제외 인구 = -49680
서울 제외 인구 = -49761
서울 제외 인구 = -49808
서울 제외 인구 = -49831
ok


In [25]:
import pandas as pd
# 엑셀 파일 열기 --- (※1)
filename = "stat_104102.xls" # 파일 이름
sheet_name = "Sheet0" # 시트 이름
book = pd.read_excel(filename, sheet_name=sheet_name, header=2) # 첫 번째 줄부터 헤더
# 2019년 인구로 정렬 --- (※2)
print(book)
#book = book.sort_values(by=['2019']) # 2019년 기준, 오름 차순
book = book.sort_values(by=['2019'], ascending=False) # 2019년 기준, 내림 차순
print(book)

   Unnamed: 0                       2011    2012    2013    2014    2015  \
0           계                     50,734  50,948  51,141  51,328  51,529   
1          서울                     10,250  10,195  10,144  10,103  10,022   
2          부산                      3,551   3,538   3,528   3,519   3,513   
3          대구                      2,508   2,506   2,502   2,493   2,487   
4          인천                      2,801   2,844   2,880   2,903   2,925   
5          광주                      1,463   1,469   1,473   1,476   1,472   
6          대전                      1,516   1,525   1,533   1,532   1,518   
7          울산                      1,136   1,147   1,156   1,166   1,173   
8          세종                          -     113     122     156     210   
9          경기                     11,937  12,093  12,235  12,358  12,522   
10         강원                      1,536   1,539   1,542   1,544   1,549   
11         충북                      1,563   1,566   1,573   1,579   1,583   
12         충

In [6]:
# pip install scikit-learn scipy matplotlib scikit-image
# pip install pandas

from sklearn import svm
# XOR의 계산 결과 데이터 --- (※1) 다르면 참 같으면 거짓
xor_data = [   
    #P, Q, result
    [0, 0, 0],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0]
]
# 학습을 위해 데이터와 레이블 분리하기 --- (※2)
data = []   # 학습 시키기 위한 데이터 변수
label = []   # 정답 레이블 변수
for row in xor_data: # scikit-learn의 머신러닝을 수행 할 때 사용하는 fit()메서드의 매개변수를 맞추기 위해서
    p = row[0]
    q = row[1]
    r = row[2]
    data.append([p, q])
    label.append(r)

print(data)
print(label)
# 데이터 학습시키기 --- (※3)
clf = svm.SVC() # SVC알고리즘을 사용하여 머신너링 수행하기 위한 객체 생성
clf.fit(data, label) # 데이터 학습하기 , 데이터와 답을 머신너링에 제시
# 데이터 예측하기 --- (※4)
pre = clf.predict(data)  # 학습한 결과를 기반으로 데이터를 예측. 
# predict()메서드에 예측하고 싶은 데이터 배열을 전달하면 데이터 수 만큼 예측결과 리턴 
print("예측결과:", pre)
# 결과 확인하기 --- (※5) 예측한 결과가 정답과 일치하는 지 확인
ok = 0; total = 0
for idx, answer in enumerate(label):
    p = pre[idx]
    if p == answer: ok += 1
    total += 1
print("정답률:", ok, "/", total, "=", ok/total)

[[0, 0], [0, 1], [1, 0], [1, 1]]
[0, 1, 1, 0]
예측결과: [0 1 1 0]
[0, 1, 1, 0]
정답률: 4 / 4 = 1.0


In [ ]:
# pandas를 이용하기.
import pandas as pd
from sklearn import svm, metrics
# XOR 연산
xor_input = [
    [0, 0, 0],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0]
]
# 입력을 학습 전용 데이터와 테스트 전용 데이터로 분류하기 --- (※1)
xor_df = pd.DataFrame(xor_input)
xor_data  = xor_df.loc[:,0:1] # 데이터
xor_label = xor_df.loc[:,2]   # 레이블
# 데이터 학습과 예측하기 --- (※2)
clf = svm.SVC()
clf.fit(xor_data, xor_label)
pre = clf.predict(xor_data)
# 정답률 구하기 --- (※3)
ac_score = metrics.accuracy_score(xor_label, pre)
print("정답률 =", ac_score)

In [12]:
# https://github.com/shivanand217/Iris-flower-dataset/blob/master/iris.csv

from sklearn import svm, metrics
import random, re
# 붓꽃의 CSV 데이터 읽어 들이기 --- (※1)
csv = []
with open('iris.csv', 'r', encoding='utf-8') as fp:
    # 한 줄씩 읽어 들이기
    for line in fp:
        line = line.strip()    # 줄바꿈 제거
        cols = line.split(',') # 쉼표로 자르기
        # 문자열 데이터를 숫자로 변환하기
#         print(cols)
#         print("*" * 30)
        fn = lambda n : float(n) if re.match(r'^[0-9\.]+$', n) else n
                        #문자열을 실수로 변환
        cols = list(map(fn, cols))
#         print(cols) 
#         print("==" * 30)
        csv.append(cols)
#print(csv)
del csv[0] # 가장 앞 줄의 헤더 제거
# 데이터 셔플하기(섞기) --- (※2)
random.shuffle(csv)  # 데이터 섞기
# 학습 전용 데이터와 테스트 전용 데이터 분할하기(2:1 비율) --- (※3)
total_len = len(csv) # 150
# print(total_len)
train_len = int(total_len * 2 / 3) # 100
# print(train_len) 
train_data = []
train_label = []
test_data = []
test_label = []
for i in range(total_len):
    data  = csv[i][0:4]
    label = csv[i][4]
    if i < train_len:
        train_data.append(data)
        train_label.append(label)
    else:
        test_data.append(data)
        test_label.append(label)
# 데이터를 학습시키고 예측하기 --- (※4)
clf = svm.SVC()
clf.fit(train_data, train_label) # 학습 시키기
pre = clf.predict(test_data) # 테스트 데이터를 적용하여 분류
# 정답률 구하기 --- (※5)
print(pre)
print(test_label)
ac_score = metrics.accuracy_score(test_label, pre) 
print("정답률 =", ac_score)

['Iris-setosa' 'Iris-virginica' 'Iris-virginica' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica' 'Iris-virginica'
 'Iris-virginica' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-virginica' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica']
['Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor'

In [14]:
# scikit-learn은 훈련용과 테스트용을 분류할 수 있는 train_test_split() 메서드를 제공하고 있다.
import pandas as pd
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
# 붓꽃의 CSV 데이터 읽어 들이기 --- (※1)
csv = pd.read_csv('iris.csv')
# 필요한 열 추출하기 --- (※2)
csv_data = csv[["SepalLength","SepalWidth","PetalLength","PetalWidth"]]
csv_label = csv["Name"]
# 학습 전용 데이터와 테스트 전용 데이터로 나누기 --- (※3)

train_data, test_data, train_label, test_label = train_test_split(csv_data, csv_label)
# 데이터 학습시키고 예측하기 --- (※4)
clf = svm.SVC()
clf.fit(train_data, train_label)
pre = clf.predict(test_data)
# 정답률 구하기 --- (※5)
ac_score = metrics.accuracy_score(test_label, pre)
print("정답률 =", ac_score)

정답률 = 1.0


In [1]:
## http://yann.lecun.com/exdb/mnist

import urllib.request as req
import gzip, os, os.path
savepath = "./mnist"
baseurl = "http://yann.lecun.com/exdb/mnist"
files = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"]
# 다운로드
if not os.path.exists(savepath): os.mkdir(savepath)
for f in files:
    url = baseurl + "/" + f
    loc = savepath + "/" + f
    print("download:", url)
    if not os.path.exists(loc):
        req.urlretrieve(url, loc)
# GZip 압축 해제
for f in files:
    gz_file = savepath + "/" + f
    raw_file = savepath + "/" + f.replace(".gz", "")
    print("gzip:", f)
    with gzip.open(gz_file, "rb") as fp:
        body = fp.read()
        with open(raw_file, "wb") as w:
            w.write(body)
print("ok")

download: http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
gzip: train-images-idx3-ubyte.gz
gzip: train-labels-idx1-ubyte.gz
gzip: t10k-images-idx3-ubyte.gz
gzip: t10k-labels-idx1-ubyte.gz
ok


In [36]:
import struct
def to_csv(name, maxdata):
    # 레이블 파일과 이미지 파일 열기
    lbl_f = open("./mnist/"+name+"-labels-idx1-ubyte", "rb")
    img_f = open("./mnist/"+name+"-images-idx3-ubyte", "rb")
    csv_f = open("./mnist/"+name+".csv", "w", encoding="utf-8")
    # 헤더 정보 읽기 --- (※1) 데이터는 리틀 엔디안으로 저장. struct모듈로 리틀엔디안을 읽어 들인다. 이때 ">"기호를 지정
    #                    8바이트에 이미지 크기의 정보가 들어 있다.
    mag, lbl_count = struct.unpack(">II", lbl_f.read(8)) # 레이블 정보
    mag, img_count = struct.unpack(">II", img_f.read(8)) # 이미지 정보
    rows, cols = struct.unpack(">II", img_f.read(8)) 
    pixels = rows * cols
    # 이미지 데이터를 읽고 CSV로 저장하기 --- (※2)
    res = []
    for idx in range(lbl_count):
        if idx > maxdata: break
        label = struct.unpack("B", lbl_f.read(1))[0] # 숫자인 글자를 가져옴
        #print(label)
        bdata = img_f.read(pixels) # 픽셀에 해당하는 바이너리값
        #print(bdata)
        sdata = list(map(lambda n: str(n), bdata))
        #print(sdata) # 빅셀의 바이너리 값을 리스트로
        # cvs파일에 저장
        csv_f.write(str(label)+",")  
        csv_f.write(",".join(sdata)+"\r\n")
        # 잘 저장됐는지 이미지 파일로 저장해서 테스트하기 -- (※3)
        if idx < 10:
            s = "P2 28 28 255\n"
            s += " ".join(sdata)
            iname = "./mnist/{0}-{1}-{2}.pgm".format(name,idx,label)
            with open(iname, "w", encoding="utf-8") as f:
                f.write(s)
    csv_f.close()
    lbl_f.close()
    img_f.close()
# 결과를 파일로 출력하기 --- (※4)
to_csv("train", 1000)
to_csv("t10k", 500)

In [37]:
from sklearn import model_selection, svm, metrics
# CSV 파일을 읽어 들이고 가공하기 --- (※1)
def load_csv(fname):
    labels = []
    images = []
    with open(fname, "r") as f:
        for line in f:
            cols = line.split(",")
            if len(cols) < 2: continue
            labels.append(int(cols.pop(0)))
            vals = list(map(lambda n: int(n) / 256, cols))
            images.append(vals)
    return {"labels":labels, "images":images}
data = load_csv("./mnist/train.csv")
test = load_csv("./mnist/t10k.csv")
# 학습하기 --- (※2)
clf = svm.SVC()
clf.fit(data["images"], data["labels"])
# 예측하기 --- (※3)
predict = clf.predict(test["images"])
# 결과 확인하기 --- (※4)
ac_score = metrics.accuracy_score(test["labels"], predict)
cl_report = metrics.classification_report(test["labels"], predict)
print("정답률 =", ac_score)
print("리포트 =")
print(cl_report)

정답률 = 0.9792
리포트 =
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.97      0.98      1032
           3       0.97      0.99      0.98      1010
           4       0.98      0.98      0.98       982
           5       0.99      0.98      0.98       892
           6       0.99      0.99      0.99       958
           7       0.98      0.97      0.97      1028
           8       0.97      0.98      0.97       974
           9       0.97      0.96      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

